# OLS Factor Selection Case Study and Multiple testing

This is based on an [article](https://www.venn.twosigma.com/vennsights/article/factor-selection-case-study) by Venn by TWO SIGMA, where relaxed Lasso regression is used for factor selection.

_"We construct 7 explanatory factors of 36 data points each. The factors are completely random values within a normal distribution with a mean of zero and a standard deviation of 2%.
We constructed a “noise” data stream that also has 36 data points with random values pulled from a normal distribution with a mean of zero and a standard deviation of 0.5%.
We constructed a dependent return stream using 100% of Factor 1 + 50% of Factor 2 + 30% of Factor 3 + “noise” data. The dependent return stream has no relationship to Factors 4 through 7."_


Below we fit the observed asset returns to the observed factors, calculate the T-statistic for each factor parameter at the 5% confidence level. This amounts to performing the hypothesis test

$ \begin{align}\mathcal{H}_0: \beta_i = 0 \\ \mathcal{H}_1: \beta_i \neq 0 \end{align}$

for each factor $i=1,...,7$.

We then evaluate the risk of false positives (type I errors), and false negatives (type II errors).

## 1. Create factors and the asset

In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm

nfactors = 7
nsamples = 36

factor_vol = 0.02 * np.eye(nfactors)
noise_vol = 0.005

factor_dist = stats.multivariate_normal(cov=factor_vol**2)
noise_dist = stats.multivariate_normal(cov=noise_vol**2)

# Asset concisting of liner combinations of factors.
weights = np.array([1.0, 0.5, 0.30, 0.0, 0.0, 0.0, 0.0])

# True hidden asset returns.
asset = lambda factor_innov: factor_innov @ weights

# Observed noisy asset returns.
noisy_asset = lambda factor_innov: asset(factor_innov) + noise_dist.rvs(factor_innov.shape[0])


## 2. Naive factor hypothesis testing

In [2]:
# Fit OLS to noisy_asset, return pvalues.
def ols_pvalues(factor_innov, asset_innov):
    X = sm.add_constant(factor_innov) # Don't forget that intercept (you should never really omit it).
    Y = asset_innov
    mdl = sm.OLS(Y, X)
    result = mdl.fit()
    return result.pvalues

# Simulate OLS pvalues.
def simulate_ols_pvalues(nsim):
    pvalues = np.zeros((nsim, 1 + nfactors))
    for i in range(nsim):
        factor_innov = factor_dist.rvs(nsamples)
        asset_innov = noisy_asset(factor_innov)
        pvalues[i] = ols_pvalues(factor_innov, asset_innov)
    return pvalues


import inference
np.random.seed(1)
nsim = 2000
pvals = simulate_ols_pvalues(nsim)
alpha = 0.05
H0 = (weights == 0.0)
stats, _ = inference.multitesting(alpha, pvals[:,1::], H0)
print('{:.2f}% risk of including at least one incorrect factor (FWER).'.format(100*stats['FWER']))
print('{:.2f}% of included factors are incorrect (FDR).'.format(100*stats['FDR']))
print('{:.2f}% risk of excluding a correct factor (FNR).'.format(100*stats['FNR']))


18.30% risk of including at least one incorrect factor (FWER).
4.91% of included factors are incorrect (FDR).
0.00% risk of excluding a correct factor (FNR).


The simulation results an 18% risk of including at least one of the "incorrect" factors 4-7. That is, we have a 18% risk of rejecting at least one true null hypothesis. Our chosen 5% significant level is the maximum risk of rejecting a true null hypthesis for one parameter. The more hypotheses tests we perform with true null hypotheses, the higher the risk we face of false rejections.

In fact, since we have four factors, each with a true null hypothesis, if we assume independent tests, we will include at least one "false" factor on average $1-(1-\alpha)^4 = 18.5\%$ of the time. However, since in reality, we do not know the truth of the seven hypotheses, we must expect to falsely including at least one factor around $1-(1-\alpha)^7 = 1/3$ of the times.
Hence, when performing multiple hypotheses tests, we must chose a more conservative confidence if we want to avoid false positives.



## 3. Multiple factor hypothesis testing

Assume that we have $m$ __independent__ hypotheses tests with corresponding p-values, where the null hypotheses are known a priori to be true.
Then the probability of observing no rejections is $(1-\alpha)^n$, and hence the probability of observing at least one false rejection is $1-(1-\alpha)^m$.

We call this confidence level for the 'family' of hypotheses tests the family-wise error rate (FWER). It can be seen as extending the usual $\alpha$ value for multiple tests. Hence, if we want a combined confidence level of $\alpha$, we could modify our original confidence level to give a maximum 5% risk of rejecting at least one true null hypothesis. Equivalently, we can modify the p-value directly.

The above procedure is called the Šidák correction, and it aims to control the probability of false discoveries, or type I error of rejecting at least one true null hypothesis. However, it is important to understand that lowering the confidence level means a sacrifice of statistical power, an increased risk of false negatives, or type II errors. Also, since the Šidák correction assumes independence, it will be conservative for tests that are positively dependent, and liberal for negatively dependent tests. Therefore, there are many other more or less powerful corrections, with various assumption on independence.

While some corrections aims at controlling the probability (FWER) of false positives, others aims at controling the expected proportion of false positives, the false discovery rate (FDR). Per definition, FWER methods are more conservative with the trade-off of lower statistical power than FDR preocedures.

### 3.1. Family-wise error rate

$FWER = P(\text{"reject at least one true null hypothesis"}) = P(N_{FP} > 0)$,

where $N_{FP}$ is the number of false positives in the family of tests.

Learn more @  https://en.wikipedia.org/wiki/Family-wise_error_rate#Controlling_procedures

[Read more...](https://en.wikipedia.org/wiki/Family-wise_error_rate#Controlling_procedures)

#### 3.1.1. Bonferroni correction

For each hypothesis test:
reject $\mathcal{H}_0$ if
$p_i \leq \alpha / m \iff mp_i \leq \alpha$.

Conservative. No assumptions. Also controls the FDR.

[Read more...](https://en.wikipedia.org/wiki/Bonferroni_correction)

#### 3.1.2. Šidák correction

For each hypothesis test:
reject $\mathcal{H}_0$ if
$p_i \leq 1 - (1-\alpha)^{1/n} \iff 1- (1-p_i)^n \leq \alpha$.

More powerful than Benferroni. Assumes independence, too liberal when negative dependence among hypotheses.

[Read more...](https://en.wikipedia.org/wiki/%C5%A0id%C3%A1k_correction)

#### 3.1.3. Holm–Bonferroni method

Order the p-values $p_{(1)}, ..., p_{(m)}$ with associate hypotheses $\mathcal{H}_{(1)}, ..., \mathcal{H}_{(m)}$.

Let $\min\{k \in \mathbb{N}\ | p_{(k)} \leq \alpha / (m+1-k)\}$.

Reject $\mathcal{H}_{(1)}, ..., \mathcal{H}_{(k-1)}$, and do not reject the rest.

At least as powerful as Bonferroni. No assumptions.

[Read more...](https://en.wikipedia.org/wiki/Holm%E2%80%93Bonferroni_method)

### 3.2. False discovery rate

$FDR = E(\text{"proportion of rejections that are false rejections"}) = E\left(\frac{N_{FP}}{N_{FP} + N_{TP}} > 0 \right)$.

FWER control exerts a more stringent control over false discovery compared to false discovery rate (FDR) procedures. FWER control limits the probability of at least one false discovery, whereas FDR control limits (in a loose sense) the expected proportion of false discoveries. Thus, FDR procedures have greater power at the cost of increased rates of type I errors, i.e., rejecting null hypotheses that are actually true.

Some of the methods are the Benjamini–Hochberg, and Benjamini–Yekutieli procedures.

[Read more...](https://en.wikipedia.org/wiki/False_discovery_rate)

http://jpktd.blogspot.com/2013/04/multiple-testing-p-value-corrections-in.html
https://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/18/lecture-18.pdf
(http://mezeylab.cb.bscb.cornell.edu/labmembers/documents/supplement%205%20-%20multiple%20regression.pdf)


## Let us try them out

In [3]:
import pandas as pd
dfs = []
for procedure in ['Raw', 'Bonferroni', 'Sidak', 'Holm', 'fdr_bh']: # (Holm implementation is slow)
    stats, _ = inference.multitesting(alpha, pvals[:,1::], H0, procedure=procedure)
    dfs += [ pd.DataFrame.from_dict(stats, orient='index', columns=[procedure]) ]

df = pd.concat(dfs, axis=1)
df = df.loc[['FWER', 'FDR', 'FNR']]
print(df)

           Raw  Bonferroni     Sidak      Holm    fdr_bh
FWER  0.183000    0.026500  0.028000  0.048500  0.105500
FDR   0.049075    0.006700  0.007075  0.012467  0.028550
FNR   0.000000    0.000833  0.000833  0.000667  0.000167


Benferroni, Sidak, and Holm all control the risk of false positives (FWER), to try to get it down to the $\leq 5\%$ level.
Bonferroni and Sidak are conservative at 2.8%, while Holm-Benferroni is closer to the 5% target. Notice how Benjamini/Hochberg has reduced FWER to 11%, since it does not control FWER, but rather the FDR. In terms of statitical power, which we measure in low False Negative Rate, FNR(Holm) $\leq$ FNR(Sidak) $\leq$ FNR(Bonferroni) $\leq$ FNR(fdr_bh)